In [1]:
import numpy as np
from RK_bin import *
import matplotlib.pyplot as plt
import matplotlib as mpl

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

N = 256
newcmp = []

vals = np.ones((N, 4))
vals[:, 0] = np.linspace(1,31/256,N)
vals[:, 1] = np.linspace(1,76/256,N)
vals[:, 2] = np.linspace(1,132/256,N)
newcmp.append(ListedColormap(vals))

vals = np.ones((N, 4))
vals[:, 0] = np.linspace(1,127/256,N)
vals[:, 1] = np.linspace(1,0/256,N)
vals[:, 2] = np.linspace(1,0/256,N)
newcmp.append(ListedColormap(vals))

vals = np.ones((N, 4))
vals[:, 0] = np.linspace(1,0/256,N)
vals[:, 1] = np.linspace(1,127/256,N)
vals[:, 2] = np.linspace(1,0/256,N)
newcmp.append(ListedColormap(vals))

colors = ['#1f4c84','#800000','#007f00']


In [3]:
#mpl.use("pgf")
preamble = r"""
\usepackage{bm}
\renewcommand{\vec}{\bm}
"""

mpl.use("pgf")


mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': preamble
})

In [4]:
d = 2
N = 100
n = N*d

np.random.seed(0)
θ = 1.2
U = np.array([[np.cos(θ),np.sin(θ)],[-np.sin(θ),np.cos(θ)]])
A = np.tile(U.T@np.diag([1,2.25])@U,(N,1))


sol = np.array([0,0])
b = A@sol #+ 2e-1*np.random.randn(n)/np.sqrt(n)

sol = np.linalg.lstsq(A,b,rcond=0)[0]

In [5]:
c = 1
x1,x2 = sol[0]-c,sol[0]+c
y1,y2 = sol[1]-c*1.25,sol[1]+c*1.25

xx = np.linspace(x1,x2,300)
yy = np.linspace(y1,y2,300)
f = lambda x0,x1: np.linalg.norm(b-A@np.array([x0,x1]))**2

In [6]:
fig,axs = plt.subplots(1,5,figsize=(8,1.95))

plt.subplots_adjust(wspace=0.1,hspace=0.0,\
                    left=0.05,right=0.95,top=0.95,bottom=0)

#axs = axs.reshape(-1)

for i in range(5):
    axs[i].contourf(xx,yy,np.log10(f(xx[None,:],yy[:,None])),np.linspace(-1,3,15),alpha=.4,cmap='gray')
    axs[i].contour(xx,yy,np.log10(f(xx[None,:],yy[:,None])),np.linspace(-1,3,15),linewidths=.5,alpha=.3,colors='k')
    axs[i].set_aspect('equal')
    axs[i].set_xlim(x1,x2)
    axs[i].set_ylim(y1,y2)
    
    axs[i].set_xticks([])
    axs[i].set_yticks([])

for i in range(5):
#    axs[i].scatter(sol[0],sol[1],color='w',edgecolor='k',marker='P',lw=.75)    
    axs[i].plot(sol[0],sol[1],color='k',marker='+',lw=2,ms=8,zorder=-1)    
 
x0s = [np.array([3.2/1.5,5/1.5]),np.array([-2/1.5,-3/1.5])]#,np.array([-.5,.5])]

k1 = 35

α_GD = 4e-4
α = 4e-4
β = .5

for i,x0 in enumerate(x0s):
    x_GD = HBM(A,b,k1,α_GD,0,x0=x0)
    axs[0].scatter(x_GD[:,0],x_GD[:,1],lw=.5,marker='.',c=np.arange(k1)**.5,cmap=newcmp[i])
    axs[0].plot(x_GD[:,0],x_GD[:,1],lw=.75,color=colors[i],alpha=.4,zorder=-1)

    x_HBM = HBM(A,b,k1,α,β,x0=x0)
    axs[1].scatter(x_HBM[:,0],x_HBM[:,1],lw=.5,marker='.',c=np.arange(k1)**.5,cmap=newcmp[i])
    axs[1].plot(x_HBM[:,0],x_HBM[:,1],lw=.75,color=colors[i],alpha=.4,zorder=-1)

    np.random.seed(0)
    B = 1
    x_SGD = minibatch_HBM(A,b,k1,α_GD,0,B,x0=x0)
    axs[2].scatter(x_SGD[:,0],x_SGD[:,1],lw=.5,marker='.',c=np.arange(k1)**.5,cmap=newcmp[i])
    axs[2].plot(x_SGD[:,0],x_SGD[:,1],lw=.75,color=colors[i],alpha=.4,zorder=-1)

    np.random.seed(0)
    B = 10
    x_mbSGD = minibatch_HBM(A,b,k1,α_GD,0,B,x0=x0)
    axs[3].scatter(x_mbSGD[:,0],x_mbSGD[:,1],lw=.5,marker='.',c=np.arange(k1)**.5,cmap=newcmp[i])
    axs[3].plot(x_mbSGD[:,0],x_mbSGD[:,1],lw=.75,color=colors[i],alpha=.4,zorder=-1)

    np.random.seed(0)
    B = 10
    x_mbHBM = minibatch_HBM(A,b,k1,α,β,B,x0=x0)
    axs[4].scatter(x_mbHBM[:,0],x_mbHBM[:,1],lw=.5,marker='.',c=np.arange(k1)**.5,cmap=newcmp[i])
    axs[4].plot(x_mbHBM[:,0],x_mbHBM[:,1],lw=.75,color=colors[i],alpha=.4,zorder=-1)


axs[0].set_title('GD')
axs[1].set_title('HBM')
axs[2].set_title('SGD/RK')
axs[3].set_title('Minibatch-SGD')
axs[4].set_title('Minibatch-HBM')

plt.savefig('imgs/2d_compare.pdf')
plt.close()